In [1]:
import warnings
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


import hvplot
import holoviews
import bokeh
import pandas as pd

In [2]:
print("hvplot version:", hvplot.__version__)
print("holoviews version:", holoviews.__version__)
print("bokeh version:", bokeh.__version__)
print("pandas version:", pd.__version__)

hvplot version: 0.11.0
holoviews version: 1.19.1
bokeh version: 3.4.3
pandas version: 2.1.4


In [4]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# make sure there is any missing values or non-numeric data
df_market_data = df_market_data.dropna() 

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [5]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [8]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

### Prepare the Data

In [91]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']])
data_scaled[0:3]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051]])

In [92]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(data_scaled, columns=df_market_data.columns)

# Copy the crypto names from the original DataFrame
df_scaled["coin_id"] = df_market_data.index

# Set the coin_id column as index
df_scaled = df_scaled.set_index("coin_id")

# Display the scaled DataFrame
print(df_scaled.head())

              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

---

### Find the Best Value for k Using the Original Data.

In [93]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))
print(k_values)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [94]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for i in k_values:  

# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=i, random_state=0)
# 2. Fit the model to the data using `df_market_data_scaled`
    kmeans.fit(df_scaled)  
# 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans.inertia_)

print(inertia_values)

[287.0, 195.82021818036043, 123.19048183836956, 79.02243535120975, 65.302379141625, 52.88851821977532, 43.91469044757746, 37.51703249010357, 32.485240832183536, 28.22289929060893, 24.545512769147678]


In [95]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,  
    'inertia': inertia_values          
}


# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

elbow_df.head()


,k_values,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.302379


In [96]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
e_plot1 = elbow_df.hvplot.line(
    x='k_values',
    y='inertia',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    width=800,
    height=400,
    rot=0
)
e_plot1

:Curve   [k_values]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for K is 4 because it gives a good balance of clusters without overcomplicating things.


---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [97]:
# Initialise the K-Means model using the best value for k
best_k = 4
kmeans_model = KMeans(n_clusters=best_k, random_state=0)

In [98]:
# Fit the K-Means model using the scaled DataFrame
kmeans_model.fit(data_scaled)

KMeans(n_clusters=4, random_state=0)

In [99]:
# Predict the clusters to group the cryptocurrencies using the scaled data
kmeans_predictions = kmeans_model.predict(data_scaled)

# Print the resulting array of cluster values.
kmeans_predictions

array([0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 0, 3, 3, 2, 3, 3, 3, 3])

In [100]:
# Create a copy of the scaled DataFrame
df_prediction = df_scaled.copy()

In [101]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_prediction['Cluster'] = kmeans_predictions  

# Display the copy of the scaled DataFrame
print(df_prediction.head())  

              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

In [102]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
c_plot1 = scatter_plot = df_prediction.hvplot.scatter(
    x="price_change_percentage_24h",  
    y="price_change_percentage_7d",    
    c="cluster",                        
    cmap='viridis',                    
    hover_cols=['coin_id'],            
    width=800,                          
    height=400,
    legend='top'  
)

# Display the scatter plot
scatter_plot

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimise Clusters with Principal Component Analysis.

In [103]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [104]:
# Use the PCA model with `fit_transform` to reduce the original scaled Dataframe
# down to three principal components.
pca_components = pca_model.fit_transform(df_scaled)  

# Reset the index to bring coin_id back as a column  
df_scaled_reset = df_scaled.reset_index()  

pca_df = pd.DataFrame(data=pca_components , columns=['PC1', 'PC2', 'PC3'])

# Add the coin_id column to the DataFrame  
pca_df['coin_id'] = df_scaled_reset['coin_id']  

# Reorder the columns to have 'coin_id' first  
pca_df = pca_df[['coin_id', 'PC1', 'PC2', 'PC3']]   

# Display the DataFrame without the default index  
print(pca_df.head(10).to_string(index=False))   

        coin_id       PC1       PC2       PC3
        bitcoin -0.600667  0.842760  0.461595
       ethereum -0.458261  0.458466  0.952877
         tether -0.433070 -0.168126 -0.641752
         ripple -0.471835 -0.222660 -0.479053
   bitcoin-cash -1.157800  2.041209  1.859715
    binancecoin -0.516534  1.388377  0.804071
      chainlink -0.450711  0.517699  2.846143
        cardano -0.345600  0.729439  1.478013
       litecoin -0.649468  0.432165  0.600303
bitcoin-cash-sv -0.759014 -0.201200 -0.217653


In [105]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca_model .explained_variance_ratio_
explained_variance

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer: 0.8950** 

In [106]:
# add the arrays together
sum(pca_model.explained_variance_ratio_)

0.8950316570309841

In [107]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(data=pca_components, columns=['PC1', 'PC2', 'PC3'])  

# Copy the crypto names from the original scaled DataFrame
pca_df['coin_id'] = df_scaled.index   

# Set the coin_id column as index
pca_df.set_index('coin_id', inplace=True)  


# Display the scaled PCA DataFrame
print(pca_df.head())

                   PC1       PC2       PC3
coin_id                                   
bitcoin      -0.600667  0.842760  0.461595
ethereum     -0.458261  0.458466  0.952877
tether       -0.433070 -0.168126 -0.641752
ripple       -0.471835 -0.222660 -0.479053
bitcoin-cash -1.157800  2.041209  1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [108]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [109]:
# Create an empty list to store the inertia values

inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# Inside the loop:

    # 1. Create a KMeans model using the loop counter for the n_clusters
  kmeans_model = KMeans(n_clusters=k, random_state=0)
    # 2. Fit the model to the data using `df_market_data_pca`
  kmeans_model.fit(pca_df)
    # 3. Append the model.inertia_ to the inertia list
  inertia_values.append(kmeans_model.inertia_)

In [117]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,            
    'inertia': inertia_values        
}
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

print(elbow_df)

    k_values     inertia
0          1  256.874086
1          2  165.901994
2          3   93.774626
3          4   49.665497
4          5   37.878747
5          6   27.618972
6          7   21.182776
7          8   17.389823
8          9   13.593650
9         10   10.559358
10        11    8.007418


In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
 
e_plot2 = elbow_df.hvplot.line(
    x='k_values', 
    y='inertia', 
    title='Elbow Curve',
    xlabel='Number of Clusters (k)', 
    ylabel='Inertia',
    width=800, 
    height=400
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**The best value for k when using the PCA data is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Yes, it differs from the best k value found using the original data, which may also be 4 or a different number depending on the analysis.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [ ]:
# Initialise the K-Means model using the best value for k
best_k = 4  # Best number of clusters based on the Elbow method
kmeans_model = KMeans(n_clusters=best_k, random_state=0)

In [ ]:
# Fit the K-Means model using the scaled PCA DataFrame
kmeans_model.fit(pca_components)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
cluster_assignments = kmeans_model.predict(pca_components)

# Print the resulting array of cluster values.
print(cluster_assignments)

In [ ]:
# Create a DataFrame with the PCA data (assuming pca_df_copy was created from PCA data)
pca_df_copy = pd.DataFrame(pca_components)

# Add the coin IDs from the original DataFrame as the first column
pca_df_copy.insert(0, 'Coin_ID', df_market_data.index)

# Rename the PCA columns
pca_df_copy.columns = ['Coin_ID', 'PC1', 'PC2', 'PC3']

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
pca_df_copy['Cluster'] = cluster_assignments

# Display the updated copy of the PCA DataFrame
print(pca_df_copy)

In [115]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
c_plot2 = pca_df_copy.hvplot.scatter(
    x='PC1',
    y='PC2',
    c='Cluster',  # Color by cluster
    hover_cols=['Coin_ID'],  # Show coin IDs on hover
    title='PCA of Cryptocurrencies',
    xlabel='Principal Component 1 (PC1)',
    ylabel='Principal Component 2 (PC2)',
    width=800,
    height=400,
    size=10
)

# Display the scatter plot
scatter_plot

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [112]:
# Composite plot to contrast the Elbow curves
combined_elbow_plot = e_plot1 + e_plot2
combined_elbow_plot

:Layout
   .Curve.I  :Curve   [k_values]   (inertia)
   .Curve.II :Curve   [k_values]   (inertia)

In [116]:
# Composite plot to contrast the clusters
combined_cluster_plot = c_plot1 + c_plot2
combined_cluster_plot

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .Scatter.II :Scatter   [PC1]   (PC2,Cluster,Coin_ID)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features in K-Means clustering may lead to less accurate clusters, as important information is lost, potentially resulting in misleading groupings.